In [1]:
# モジュールのインポート
from matplotlib import pyplot
from matplotlib import patches
import random
import numpy
import array
import numpy

from itertools import repeat
from collections.abc import Sequence
from deap import algorithms
from deap import base
from deap import creator
from deap import tools
%matplotlib inline

In [2]:
# roomというclassを作る
class Room():
    def __init__(self,n,s,fre_move,w=0,l=0,x=0,y=0):
        self.n = n
        self.s = s
        self.fre_move = fre_move
        self.x = x
        self.y = y
        self.w = w
        self.l = l
        
    def distance(self,b):
        return (abs(self.x-b.x)+abs(self.y-b.y))*self.fre_move[b.n]
    def construction(self):
        S = self.w*self.l
        wall = (self.w+self.l)*2
        return S*1+wall*2
    def penalty(self):
        S = self.w*self.l
        return abs(S-self.s)

In [3]:
# 設定（部屋番号,部屋の面積,部屋と部屋の間の移動コスト）
Room_1 = Room(0,15,[0,1,1,1,1,1,1,1])
Room_2 = Room(1,15,[1,0,1,1,1,1,1,1])
Room_3 = Room(2,15,[1,1,0,1,1,1,1,1])
Room_4 = Room(3,15,[1,1,1,0,1,1,1,1])
Room_5 = Room(4,15,[1,1,1,1,0,1,1,1])
Room_6 = Room(5,15,[1,1,1,1,1,0,1,1])
Room_7 = Room(6,15,[1,1,1,1,1,1,0,1])
Room_8 = Room(7,15,[1,1,1,1,1,1,1,0])
Rooms = [Room_1,Room_2,Room_3,Room_4,Room_5,Room_6,Room_7,Room_8]

In [4]:
# 遺伝子を生成
def room_gene():
    gene = random.sample(range(0,8),8)# seq_rooms
    for i in range(8):# width
        gene.append(random.randint(1,10))
    for i in range(8):# length
        gene.append(random.randint(1,10))
    for i in range(2):# start_point
        gene.append(random.randint(0,10))
    for i in range(8):# y
        gene.append(random.randint(0,1))
    return gene

# 評価関数
def Eval(individual):
    seq = individual[0:8]
    width = individual[8:16]
    length = individual[16:24]
    start_point = individual[24:26]
    up_or_down = individual[26:34]
    
    global Rooms
    for room in Rooms:
        room.w = width[room.n]
        room.l = length[room.n]
        room.y = up_or_down[room.n]
    seq_rooms = [i for _, i in sorted(zip(seq, Rooms))]
    zero = []
    one = []
    for seq_room in seq_rooms:
        
        if seq_room.y == 0:
            left_point = start_point[0]+sum(zero)
            seq_room.x = left_point+seq_room.w/2
            zero.append(seq_room.w)
        else:
            left_point = start_point[1]+sum(one)
            seq_room.x = left_point+seq_room.w/2
            one.append(seq_room.w)
    distance_cost = []
    construction_cost = []
    penalty_sum = []
    for room in Rooms:
        distance_cost.append(sum([room.distance(i) for i in Rooms]))
        construction_cost.append(room.construction())
        penalty_sum.append(room.penalty())
    dis_c = sum(distance_cost)
    con_c = sum(construction_cost)
    pen_c = sum(penalty_sum)
    a1,a2 = 2,1
    return dis_c, con_c*a1+pen_c*a2

# 交叉
def CX(ind1, ind2, indpb=0.05):
    seq1,seq2 = ind1[0:8],ind2[0:8]
    wl1, wl2= ind1[8:26], ind2[8:26]
    up_or_down1, up_or_down2 = ind1[26:34], ind2[26:34]
    
    ## cxPartialyMatched
    size1 = min(len(seq1), len(seq2))
    p1, p2 = [0] * size1, [0] * size1

    # Initialize the position of each indices in the individuals
    for i in range(size1):
        p1[seq1[i]] = i
        p2[seq2[i]] = i
    # Choose crossover points
    cxpoint1 = random.randint(0, size1)
    cxpoint2 = random.randint(0, size1 - 1)
    if cxpoint2 >= cxpoint1:
        cxpoint2 += 1
    else:  # Swap the two cx points
        cxpoint1, cxpoint2 = cxpoint2, cxpoint1

    # Apply crossover between cx points
    for i in range(cxpoint1, cxpoint2):
        # Keep track of the selected values
        temp1 = seq1[i]
        temp2 = seq2[i]
        # Swap the matched value
        seq1[i], seq1[p1[temp2]] = temp2, temp1
        seq2[i], seq2[p2[temp1]] = temp1, temp2
        # Position bookkeeping
        p1[temp1], p1[temp2] = p1[temp2], p1[temp1]
        p2[temp1], p2[temp2] = p2[temp2], p2[temp1]

    ## cxUniform
    size2 = min(len(wl1), len(wl2))
    for i in range(size2):
        if random.random() < indpb:
            wl1[i], wl2[i] = wl2[i], wl1[i]

    ## cxTwoPoint
    size3 = min(len(up_or_down1), len(up_or_down2))
    cxpoint3 = random.randint(1, size3)
    cxpoint4 = random.randint(1, size3 - 1)
    if cxpoint4 >= cxpoint3:
        cxpoint4 += 1
    else:  # Swap the two cx points
        cxpoint3, cxpoint4 = cxpoint4, cxpoint3

    up_or_down1[cxpoint3:cxpoint4], up_or_down2[cxpoint3:cxpoint4] \
        = up_or_down2[cxpoint3:cxpoint4], up_or_down1[cxpoint3:cxpoint4]
    
    ind1 = seq1+wl1+up_or_down1
    ind2 = seq2+wl2+up_or_down2
    return ind1, ind2

# 突然変異
def MUT(individual, low, up, indpb):
    seq = individual[0:8]
    wl = individual[8:26]
    up_or_down = individual[26:34]
    
    # mutShuffleIndexes
    size1 = len(seq)
    for i in range(size1):
        if random.random() < indpb:
            swap_indx = random.randint(0, size1 - 2)
            if swap_indx >= i:
                swap_indx += 1
            seq[i], seq[swap_indx] = \
                seq[swap_indx], seq[i]

    # mutUniformInt
    size2 = len(wl)
    if not isinstance(low, Sequence):
        low = repeat(low, size2)
    elif len(low) < size2:
        raise IndexError("low must be at least the size of individual: %d < %d" % (len(low), size2))
    if not isinstance(up, Sequence):
        up = repeat(up, size2)
    elif len(up) < size2:
        raise IndexError("up must be at least the size of individual: %d < %d" % (len(up), size2))

    for i, xl, xu in zip(range(size2), low, up):
        if random.random() < indpb:
            wl[i] = random.randint(xl, xu)

    # mutFlipBit
    for i in range(len(up_or_down)):
        if random.random() < indpb:
            up_or_down[i] = type(up_or_down[i])(not up_or_down[i])
    
    individual = seq+wl+up_or_down
    return individual,

In [5]:
# 定義
creator.create("FitnessMin", base.Fitness, weights=(-1.0,-1.0))
creator.create("Individual", list, fitness=creator.FitnessMin)

toolbox = base.Toolbox()

# 初期条件を構築
toolbox.register("individual", tools.initIterate, creator.Individual, room_gene)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

# 評価関数、交差、突然変異、選択淘汰
toolbox.register("evaluate", Eval)
toolbox.register("mate", CX, indpb=0.05)
toolbox.register("mutate", MUT,low=1,up=10, indpb=0.05)
toolbox.register("select", tools.selTournament, tournsize=50)

In [6]:
toolbox.register("mate", tools.cxTwoPoint)
toolbox.register("mutate", tools.mutFlipBit, indpb=0.05)
toolbox.register("select", tools.selTournament, tournsize=3)

def main():
    random.seed(64)

    pop = toolbox.population(n=300)
    hof = tools.HallOfFame(1)
    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("avg", numpy.mean)
    stats.register("std", numpy.std)
    stats.register("min", numpy.min)
    stats.register("max", numpy.max)
    

    pop, log = algorithms.eaSimple(pop, toolbox, cxpb=0.5, mutpb=0.2, ngen=40, 
                                   stats=stats, halloffame=hof, verbose=True)

    return pop, log, hof

if __name__ == "__main__":
    main()

gen	nevals	avg    	std    	min	max 
0  	300   	1012.08	438.118	277	2055


TypeError: '<' not supported between instances of 'Room' and 'Room'

In [ ]:
Best1 = [0, 1, 2, 3, 4, 5, 6, 7, 3, 3, 3, 3, 3, 3, 3, 3, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 0, 0, 0, 0, 1, 1, 1, 1]
Best = [4, 2, 3, 6, 7, 0, 1, 5, 2, 4, 2, 4, 2, 1, 4, 1, 4, 6, 4, 4, 2, 2, 7, 7, 5, 5, 0, 0, 1, 0, 1, 1, 1, 1]
seq = Best[0:8]
width = Best[8:16]
length = Best[16:24]
start_point = Best[24:26]
up_or_down = Best[26:34]
for room in Rooms:
        room.w = width[room.n]
        room.l = length[room.n]
        room.y = up_or_down[room.n]
seq_rooms = [i for _, i in sorted(zip(seq, Rooms))]
zero = []
one = []
for seq_room in seq_rooms:
    if seq_room.y == 0:
        left_point = start_point[0]+sum(zero)
        seq_room.x = left_point+seq_room.w/2
        zero.append(seq_room.w)
    else:
        left_point = start_point[1]+sum(one)
        seq_room.x = left_point+seq_room.w/2
        one.append(seq_room.w)
print(Eval(Best))
fig, ax = pyplot.subplots()
 
ax.set_xlim([0,21])
ax.set_ylim([-11,11])
ax.grid()
ax.add_patch(patches.Rectangle( xy=(0,-10.5) ,facecolor="white",edgecolor="black", width=20, height=21))
# 3. 図形オブジェクト生成
for room in Rooms:
    if room.y == 0:
        y = 1
    else:
        y = -1*room.l
    r = patches.Rectangle( xy=(room.x-room.w/2,y) ,facecolor="white",edgecolor="red", width=room.w, height=room.l) # 四角形のオブジェクト
    ax.add_patch(r)


pyplot.axis('off')
pyplot.show()